In [1]:
import json
import folium
import requests
import mimetypes
import http.client
import pandas as pd
from folium.plugins import HeatMap
from pandas.io.json import json_normalize

In [2]:
conn= http.client.HTTPSConnection("api.covid19api.com")
payload=''
headers={}
conn.request("GET","/summary",payload,headers)
res=conn.getresponse()
data=res.read().decode('UTF-8')

In [3]:
covid1 = json.loads(data)

In [4]:
pd.json_normalize(covid1['Countries'],sep=",");

In [5]:
df=pd.DataFrame(covid1['Countries'])
df;

In [6]:
covid2=df.drop(columns=['CountryCode','Slug','Date','Premium'],axis=1)
covid2;

In [7]:
 m=folium.Map(tiles="Stamen Terrain",min_zoom=1.5)
m;

In [8]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes=f'{url}/world-countries.json'

In [9]:
folium.Choropleth(
    geo_data=country_shapes,
    min_zoom=2,
    name='Covid-19',
    data=covid2,
    columns=['Country','TotalConfirmed'],
    key_on='feature.properties.name',
    fill_color='OrRd',
    nan_fill_color='black',
    legend_name='Total Confirmed Covid Cases',
).add_to(m)
m;

In [10]:
covid2.update(covid2['TotalConfirmed'].map('Total Confirmed:{}'.format))
covid2.update(covid2['TotalRecovered'].map('Total Recovered:{}'.format))

In [11]:
coordinates=pd.read_csv('https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv')
coordinates;

In [12]:
covid_final=pd.merge(covid2,coordinates,on='Country')

In [13]:
def plotDot(point):
    folium.CircleMarker(location=[point.latitude,point.longitude],
                       radius=5,
                       weight=2,
                       popup=[point.Country,point.TotalConfirmed,point.TotalRecovered],
                             fill_color='#000000').add_to(m)

In [14]:
covid_final.apply(plotDot,axis=1)
m.fit_bounds(m.get_bounds())
m;

In [15]:
m1= folium.Map(tiles='StamenToner',min_zoom=2)
m1;

In [16]:
deaths=covid_final['TotalDeaths'].astype(float)

In [17]:
lat=covid_final['latitude'].astype(float)

In [18]:
lon=covid_final['longitude'].astype(float)

In [19]:
m1.add_child(HeatMap(zip(lat,lon,deaths),radius=0));

### HEAT MAP OF COVID TRANSMISSION

In [20]:
def plotDot(point):
    folium.CircleMarker(location=[point.latitude,point.longitude],
                       radius=5,
                       weight=2,
                       popup=[point.Country,point.TotalConfirmed,point.TotalRecovered],
                             fill_color='#000000').add_to(m1)
covid_final.apply(plotDot,axis=1)
m1.fit_bounds(m1.get_bounds())
m1